<h2> Load Data</h2>

In [30]:
import warnings
import matplotlib.pyplot as plt
import preparers
import json
import numpy as np
import pandas as pd
import os
import preparers
import sonaion_analysis as son
import GenSnippetsLib as snippet
from pandas.core.common import SettingWithCopyWarning
from PIL import Image

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [31]:
display_width = 1920
display_height = 1080
denoise_degree = 10
left_color = (0.0, 1.0, 1.0)
right_color = (0.0, 1.0, 0.0)

In [23]:
df_psycho, df_eyetracking = preparers.load_raw(1)

In [32]:
def create_3_snippet(top_right, bottom_right, left, display_width, display_height):
    bg = snippet.create_background(display_width, display_height, (180, 180, 180, 255))
    top_right = Image.open(top_right)
    bottom_right = Image.open(bottom_right)
    left = Image.open(left)

    img = snippet.place_image_on(bg, np.array(left), 0.3, 0.5)
    img = snippet.place_image_on(img, np.array(top_right), 0.7, 0.7)
    img = snippet.place_image_on(img, np.array(bottom_right), 0.7, 0.3)

    return img

def create_1_snipper(image, display_width, display_height):
    bg = snippet.create_background(display_width, display_height, (180, 180, 180, 255))
    image = Image.open(image)

    img = snippet.place_image_on(bg, np.array(image), 0.5, 0.5)
    return img

In [33]:
images = {}
images["BE"] = create_3_snippet("./images/Brokantiny.PNG", "./images/BrocantinyErklärung.png", "./images/BrocantinyBeispiel.png", display_width, display_height)
images["B1"] = create_3_snippet("./images/Brokantiny.PNG", "./images/BrocantinyErklärung.png", "./images/Brocantiny1.png", display_width, display_height)
images["B2"] = create_3_snippet("./images/Brokantiny.PNG", "./images/BrocantinyErklärung.png", "./images/Brocantiny2.png", display_width, display_height)
images["B3"] = create_3_snippet("./images/Brokantiny.PNG", "./images/BrocantinyErklärung.png", "./images/Brocantiny3.png", display_width, display_height)
images["B4"] = create_3_snippet("./images/Brokantiny.PNG", "./images/BrocantinyErklärung.png", "./images/Brocantiny4.png", display_width, display_height)

images["PE"] = create_3_snippet("./images/Brokantiny.PNG", "./images/ProgrammErklärung.png", "./images/ProgrammErklärung.png", display_width, display_height)
images["P1"] = create_3_snippet("./images/Brokantiny.PNG", "./images/ProgrammErklärung.png", "./images/Programm1.png", display_width, display_height)
images["P2"] = create_3_snippet("./images/Brokantiny.PNG", "./images/ProgrammErklärung.png", "./images/Programm2.png", display_width, display_height)
images["P3"] = create_3_snippet("./images/Brokantiny.PNG", "./images/ProgrammErklärung.png", "./images/Programm3.png", display_width, display_height)
images["P4"] = create_3_snippet("./images/Brokantiny.PNG", "./images/ProgrammErklärung.png", "./images/Programm4.png", display_width, display_height)

images["S1"] = create_1_snipper("./images/Satz1.png", display_width, display_height)
images["S1"] = create_1_snipper("./images/Satz2.png", display_width, display_height)
images["S1"] = create_1_snipper("./images/Satz3.png", display_width, display_height)

In [34]:
def update_names(df_eyetracking, name_dict):
    df_eyetracking = df_eyetracking.rename(columns=name_dict)
    return df_eyetracking

In [35]:
def update_to_display_coordinates(df_eyetracking):
    df_eyetracking["display_x"] = df_eyetracking["display_x"].apply(lambda x: x * display_width)
    df_eyetracking["display_x"] = df_eyetracking["display_x"].astype(float).fillna(0).astype(int)

    df_eyetracking["display_y"] = df_eyetracking["display_y"].apply(lambda x: x * display_height)
    df_eyetracking["display_y"] = df_eyetracking["display_y"].astype(float).fillna(0).astype(int)
    return df_eyetracking

In [62]:
def update_normalize_time(df_eyetracking):
    df_eyetracking["time"] = df_eyetracking["time"] - df_eyetracking["time"][0]
    return df_eyetracking

In [63]:
def update_replace_invalid(df_eyetracking):
    (
        replaced_x,
        replaced_y,
        _replaced_diameter,
        replaced_valid,
    ) = son.eyetracking.preprocessing_invalid.replace_with_prev_invalid(
        df_eyetracking["display_x"],
        df_eyetracking["display_y"],
        df_eyetracking["display_x"],
        df_eyetracking["valid"],
    )

    df_eyetracking["display_x"] = replaced_x
    df_eyetracking["display_y"] = replaced_y
    df_eyetracking["valid"] = replaced_valid
    return df_eyetracking

In [64]:
def update_denoise(df_eyetracking, denoise_degree):
    for _i in range(denoise_degree):
        denoised_x, denoised_y = son.eyetracking.preprocessing_denoise.denoise(
            df_eyetracking["display_x"],
            df_eyetracking["display_y"],
            son.eyetracking.preprocessing_denoise.parabola_5_kernel,
        )

        df_eyetracking["display_x"] = denoised_x
        df_eyetracking["display_y"] = denoised_y
    return df_eyetracking

In [65]:
def update_match_to_line(df_eyetracking, line_middle, cat):
    df_eyetracking["display_y"] =  df_eyetracking["display_y"].apply(lambda x: line_middle[cat[x]] if cat[x] is not None else float("Nan"))
    df_eyetracking = df_eyetracking[df_eyetracking["display_y"].notnull()]
    df_eyetracking["display_y"] = df_eyetracking["display_y"].astype(int)
    df_eyetracking = df_eyetracking.reset_index()
    return df_eyetracking

In [66]:
def mask_function():
    return lambda height, width, coordinate: son.utils.masks.create_circular_mask(height, width, coordinate, 30)

def create_heatmap(df_eyetracking, color, width, height):
    heat_data = son.eyetracking.heatmap.create_heatmap(
        df_eyetracking["display_x"],
        df_eyetracking["display_y"],
        df_eyetracking["valid"],
        0.004,
        height,
        width,
        mask_function(),
    )

    heat_data = heat_data / heat_data.max()
    heat_data = np.array([heat_data * color[0], heat_data * color[1], heat_data * color[2], heat_data])
    heat_data = np.moveaxis(heat_data, 0, 2)
    return heat_data

In [67]:
def create_sequence_diagram(df_eyetracking, color, width, height, max_pixel):
    step_size = max_pixel/len(df_eyetracking)

    sequence_data = son.eyetracking.sequence.create_sequence_diagram_y(
        df_eyetracking["display_y"],
        df_eyetracking["valid"],
        height,
        width,
        offset=0,
        step=step_size,
        should_skip=False,
    )
    sequence_data = np.array([sequence_data * color[0], sequence_data * color[1], sequence_data * color[2], sequence_data])
    sequence_data = np.moveaxis(sequence_data, 0, 2)
    return sequence_data

In [70]:
left_naming = {
    "l_valid" : "valid",
    "l_display_x" : "display_x",
    "l_display_y" : "display_y",
}

right_naming = {
    "r_valid" : "valid",
    "r_display_x" : "display_x",
    "r_display_y" : "display_y",
}

for idx in range(1, len(df_psycho)):
    name = df_psycho["Snippet"][idx]
    image = images[name]
    df_eye = df_eyetracking[(df_eyetracking["time"] >= df_psycho["SnippetStart"][idx]) & (df_eyetracking["time"] < df_psycho["SnippetStop"][idx])]
    print("Working with", name, "...")
    for naming, color, eye in zip([left_naming, right_naming], [left_color, right_color], ["Left Eye", "Right Eye"]):
        print("\t", eye, "...")
        df_eye_tmp = df_eye.reset_index(drop=True)
        df_eye_tmp = update_names(df_eye_tmp, naming)
        df_eye_tmp = update_to_display_coordinates(df_eye_tmp)
        df_eye_tmp = update_normalize_time(df_eye_tmp)
        df_eye_tmp = update_replace_invalid(df_eye_tmp)
        df_eye_tmp = update_denoise(df_eye_tmp, denoise_degree)

        heat_mask = create_heatmap(df_eye_tmp, color, display_width, display_height)
        sequence_mask = create_sequence_diagram(df_eye_tmp, color, display_width, display_height, 500)

        snippet_folder = "./results/" + name + "/"
        if not os.path.exists(snippet_folder):
            os.makedirs(snippet_folder)


        plt.imshow(image, zorder=0)
        plt.imshow(heat_mask, zorder=1)
        plt.savefig(snippet_folder + "Heatmap_" + eye + ".pdf", dpi = 300)
        plt.close()

        plt.imshow(image, zorder=0)
        plt.imshow(sequence_mask, zorder=1)
        plt.savefig(snippet_folder + "Sequence_" + eye + ".pdf", dpi = 300)
        plt.close()

Working with B1 ...
	 Left Eye ...
5.020277
	 Right Eye ...
5.020277
Working with B2 ...
	 Left Eye ...
6.236307
	 Right Eye ...
6.236307
Working with B3 ...
	 Left Eye ...
7.456341
	 Right Eye ...
7.456341
Working with B4 ...
	 Left Eye ...
8.372373
	 Right Eye ...
8.372373


KeyboardInterrupt: 